In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import pymongo
import warnings
warnings.filterwarnings("ignore")
username = "tagger-admin"
password = "tvaiadmin"
db_client = pymongo.MongoClient('104.198.62.226', username=username,
                                password=password,
                                authSource='tags',
                                authMechanism='SCRAM-SHA-256', port=27017).tags

In [2]:
import aerospike
config = {
    'hosts': [('35.228.136.58', 3000)]
}
client = aerospike.client(config).connect()
data_as_key = ("ids_to_validate", "data", "data")
tags_dictionary = client.get(data_as_key)[-1]['tags']
bert_data_key = ("ids_to_validate", "data", "bert_tags")
len(tags_dictionary)
guten_tag_labels = client.get(bert_data_key)[-1]["gutenTag"]

In [3]:
from collections import Counter

In [4]:
def show_percentage(results_val_algo):
    
    total_sum = 0
    for i in results_val_algo:
        total_sum += results_val_algo[i]
        
    results_val_algo_percent = {}
    for i in results_val_algo:
        results_val_algo_percent[i] = round(results_val_algo[i]/total_sum, 2)

    return results_val_algo_percent     

In [5]:
def show_end_result(totalResults):
    
    results_val_algo = {
        "[0, 0, 0]": 0,
        "[0, 0, 1]": 0,
        "[0, 1, 0]": 0,
        "[0, 1, 1]": 0,
        "[1, 0, 0]": 0,
        "[1, 0, 1]": 0,
        "[1, 1, 0]": 0,
        "[1, 1, 1]": 0
    }

    for row in totalResults:
        if row == [0, 0, 0]:
            results_val_algo["[0, 0, 0]"] += 1
        if row == [0, 0, 1]:
            results_val_algo["[0, 0, 1]"] += 1
        if row == [0, 1, 0]:
            results_val_algo["[0, 1, 0]"] += 1
        if row == [0, 1, 1]:
            results_val_algo["[0, 1, 1]"] += 1

        if row == [1, 0, 0]:
            results_val_algo["[1, 0, 0]"] += 1        
        if row == [1, 0, 1]:
            results_val_algo["[1, 0, 1]"] += 1

        if row == [1, 1, 0]:
            results_val_algo["[1, 1, 0]"] += 1
        if row == [1, 1, 1]:
            results_val_algo["[1, 1, 1]"] += 1

    results = {
        "count": results_val_algo,
        "percentage": show_percentage(results_val_algo)
    }

    return results

In [6]:
def dict_values(tags_list):
    tags_list = [tag.lower() for tag in tags_list]
    tags_collector = []
    for tag in tags_list:
        if tag.lower() in tags_dictionary:
            if tags_dictionary[tag.lower()] != [0]:
                tags_collector += tags_dictionary[tag.lower()]     
        else:
            tags_collector.append(tag.lower())                     
    return list(set(tags_collector))

In [7]:
def result_for_model(models, tagger, analyst):
    
    all_tags = models + tagger + analyst
    all_tags = set(all_tags)
    result = []
    for tag in all_tags:
        
        check_list = [0,0,0]
        
        if tag in models:
            check_list[0] = 1
            
        if tag in tagger:
            check_list[1] = 1
            
        if tag in analyst:
            check_list[2] = 1
            
        result.append(check_list)
    return result

In [8]:
def get_results(start_date, end_date, model_type):
    
    totalProcessed = db_client["tweets_pipeline_v2"].find({"$or": [{"status":"graphicone_search"}, {"status":"graphicone_feed"}], 
                                          'approved_timestamp': {'$gte': end_date, '$lte': start_date}
                                                          })
    totalResults = []

    for i in totalProcessed:
        guten_tag = []
        entity = []
        tagger = []
        analyst = []

        if "bert_tags" in i:
            guten_tag = dict_values(i["bert_tags"])

        if "entity_prepared" in i:            
            entity = dict_values(i["entity_prepared"])

        if "confirmed_after_validate" in i:
            tagger = dict_values(i["confirmed_after_validate"])

        if "grafeed_confirmed" in i:
            analyst = dict_values(i["grafeed_confirmed"])


        if model_type == 'gutenTag':
            models = guten_tag        
            tagger = list(set(tagger).intersection(set(guten_tag_labels)))
            analyst = list(set(analyst).intersection(set(guten_tag_labels)))


        if model_type == 'entity':
            models = entity       
            analyst = list(set(analyst) - (set(guten_tag_labels)))
            tagger = list(set(tagger) - (set(guten_tag_labels)))

        if model_type == 'all_models':
            models = entity + guten_tag       
            analyst = analyst
            tagger = tagger


        result = result_for_model(models, tagger, analyst)

        totalResults += result
        
    return totalResults

In [9]:
start_date = datetime.today()
end_date = datetime(2020, 6, 23)
model_type = "all_models"
# model_type = "entity"
# model_type = "gutenTag"
# 
results = get_results(start_date, end_date, model_type)
    
show_end_result(results)

{'count': {'[0, 0, 0]': 0,
  '[0, 0, 1]': 1086,
  '[0, 1, 0]': 1197,
  '[0, 1, 1]': 8450,
  '[1, 0, 0]': 1602,
  '[1, 0, 1]': 42289,
  '[1, 1, 0]': 331,
  '[1, 1, 1]': 11167},
 'percentage': {'[0, 0, 0]': 0.0,
  '[0, 0, 1]': 0.02,
  '[0, 1, 0]': 0.02,
  '[0, 1, 1]': 0.13,
  '[1, 0, 0]': 0.02,
  '[1, 0, 1]': 0.64,
  '[1, 1, 0]': 0.01,
  '[1, 1, 1]': 0.17}}

In [ ]:
# OБЕ МОДЕЛИ
{'count': {'[0, 0, 0]': 0,
  '[0, 0, 1]': 310,
  '[0, 1, 0]': 164,
  '[0, 1, 1]': 849,
  '[1, 0, 0]': 149,
  '[1, 0, 1]': 41,
  '[1, 1, 0]': 142,
  '[1, 1, 1]': 3011},
 'percentage': {'[0, 0, 0]': 0.0,
  '[0, 0, 1]': 0.07,
  '[0, 1, 0]': 0.04,
  '[0, 1, 1]': 0.18,
  '[1, 0, 0]': 0.03,
  '[1, 0, 1]': 0.01,
  '[1, 1, 0]': 0.03,
  '[1, 1, 1]': 0.65}}

# ТЭГЕР
{'count': {'[0, 0, 0]': 0,
  '[0, 0, 1]': 219,
  '[0, 1, 0]': 78,
  '[0, 1, 1]': 527,
  '[1, 0, 0]': 96,
  '[1, 0, 1]': 26,
  '[1, 1, 0]': 94,
  '[1, 1, 1]': 1445},
 'percentage': {'[0, 0, 0]': 0.0,
  '[0, 0, 1]': 0.09,
  '[0, 1, 0]': 0.03,
  '[0, 1, 1]': 0.21,
  '[1, 0, 0]': 0.04,
  '[1, 0, 1]': 0.01,
  '[1, 1, 0]': 0.04,
  '[1, 1, 1]': 0.58}}

# ЭНТИТИ
{'count': {'[0, 0, 0]': 0,
  '[0, 0, 1]': 91,
  '[0, 1, 0]': 87,
  '[0, 1, 1]': 341,
  '[1, 0, 0]': 171,
  '[1, 0, 1]': 15,
  '[1, 1, 0]': 46,
  '[1, 1, 1]': 1547},
 'percentage': {'[0, 0, 0]': 0.0,
  '[0, 0, 1]': 0.04,
  '[0, 1, 0]': 0.04,
  '[0, 1, 1]': 0.15,
  '[1, 0, 0]': 0.07,
  '[1, 0, 1]': 0.01,
  '[1, 1, 0]': 0.02,
  '[1, 1, 1]': 0.67}}



In [15]:
#ОБЕ МОДЕЛИ
'count': {
  '[0, 0, 0]': 0,
  '[0, 0, 1]': 733,
  '[0, 1, 0]': 1122,
  '[0, 1, 1]': 10789,
  '[1, 0, 0]': 1029,
  '[1, 0, 1]': 105,
  '[1, 1, 0]': 311,
  '[1, 1, 1]': 6695},
 'percentage': {
  '[0, 0, 0]': 0.0,
  '[0, 0, 1]': 0.04,
  '[0, 1, 0]': 0.05,
  '[0, 1, 1]': 0.52,
  '[1, 0, 0]': 0.05,
  '[1, 0, 1]': 0.01,
  '[1, 1, 0]': 0.01,
  '[1, 1, 1]': 0.32}

In [ ]:
# ENTITY
{'count': {'[0, 0, 0]': 0,
  '[0, 0, 1]': 325,
  '[0, 1, 0]': 447,
  '[0, 1, 1]': 7984,
  '[1, 0, 0]': 877,
  '[1, 0, 1]': 5,
  '[1, 1, 0]': 62,
  '[1, 1, 1]': 894},
 'percentage': {'[0, 0, 0]': 0.0,
  '[0, 0, 1]': 0.03,
  '[0, 1, 0]': 0.04,
  '[0, 1, 1]': 0.75,
  '[1, 0, 0]': 0.08,
  '[1, 0, 1]': 0.0,
  '[1, 1, 0]': 0.01,
  '[1, 1, 1]': 0.08}}

# TAGGER
{'count': 
  '[0, 0, 0]': 0,
  '[0, 0, 1]': 410,
  '[0, 1, 0]': 681,
  '[0, 1, 1]': 2992,
  '[1, 0, 0]': 812,
  '[1, 0, 1]': 98,
  '[1, 1, 0]': 243,
  '[1, 1, 1]': 5614},
 'percentage': 
  '[0, 0, 0]': 0.0,
  '[0, 0, 1]': 0.04,
  '[0, 1, 0]': 0.06,
  '[0, 1, 1]': 0.28,
  '[1, 0, 0]': 0.07,
  '[1, 0, 1]': 0.01,
  '[1, 1, 0]': 0.02,
  '[1, 1, 1]': 0.52}